In [8]:
import pandas as pd
from docx import Document 

In [9]:
# data = pd.read_excel("../Base_filtrada.xlsx")
data = pd.read_excel("../Bases/Base_filtrada_idade.xlsx")
# data = pd.read_excel("../Bases/Base_plano_Cartesiano.xlsx")

### Separação por natureza dos dados

In [10]:
colunas_categoricas = ['V0001', 'A02201', 'C004', 'C006', 'C009', 'L01701', 'L018', 'M01601', 'N010','P034',
                       'P050', 'P052', 'Q092', 'Q11006', 
                       'Q11604', 'Q074']

In [11]:
colunas_numericas = ['V0022', 'A02305', 'A02306', 'C008', 'P00104', 'P00404', 'P035', 'P03701', 'P03702', 
                     'P053', 'P05901', 'P05902', 'P05903', 'P05904']  

In [12]:
# colunas_categoricas = ['V0001', 'C004', 'C006', 'C009', 'L01701', 'L018', 'M01601', 'N010','P034|P035', 'P03701|P03702',
#                        'P050|P052', 'P05901|P05902|P05903|P05904', 'Q092', 'Q11006', 'Q11604', 'Q074']

In [13]:
# colunas_numericas = ['V0022', 'A02201|A02305|A02306', 'C008', 'P00104', 'P00404', 'P053']  

garantir que as colunas numericas sao numericas

In [14]:
for coluna in colunas_numericas:
    data[coluna] = pd.to_numeric(data[coluna], errors='coerce')

## Tabela numerica

In [15]:
estatisticas_numericas = {
    'Coluna': colunas_numericas,
    'Mínimo': [round(data[col].min(), 2) for col in colunas_numericas],
    'Máximo': [round(data[col].max(), 2) for col in colunas_numericas],
    'Média': [round(data[col].mean(), 2) for col in colunas_numericas],
    'Desvio Padrão': [round(data[col].std(), 2) for col in colunas_numericas],
    'Dados Faltantes': [data[col].isnull().sum() for col in colunas_numericas],
    '% Faltantes': [round((data[col].isnull().sum() / len(data)) * 100, 2) for col in colunas_numericas]
}
df_estatisticas_numericas = pd.DataFrame(estatisticas_numericas)

In [16]:
df_estatisticas_numericas = pd.DataFrame(estatisticas_numericas)

## Tabela Categorica

In [17]:
estatisticas_categoricas_detalhadas = {
    'Coluna': [],
    'Categoria': [],
    'Contagem': [],
    '% da Coluna': []
}

for coluna in colunas_categoricas:
    # Extrair categorias e contagens (incluindo NaN)
    contagem = data[coluna].value_counts(dropna=False).reset_index()
    contagem.columns = ['Categoria', 'Contagem']
    
    # Converter categorias para numérico (se possível) e ordenar
    try:
        contagem['Categoria_Num'] = pd.to_numeric(contagem['Categoria'], errors='coerce')
        contagem = contagem.sort_values(by='Categoria_Num').drop('Categoria_Num', axis=1)
    except:
        contagem = contagem.sort_values(by='Categoria')  # Caso não sejam números
    
    # Adicionar dados faltantes no final
    faltantes = data[coluna].isnull().sum()
    total_linhas = len(data)
    
    for _, row in contagem.iterrows():
        categoria = str(row['Categoria'])
        estatisticas_categoricas_detalhadas['Coluna'].append(coluna)
        estatisticas_categoricas_detalhadas['Categoria'].append(categoria)
        estatisticas_categoricas_detalhadas['Contagem'].append(row['Contagem'])
        estatisticas_categoricas_detalhadas['% da Coluna'].append(round((row['Contagem'] / total_linhas) * 100, 2))

# Criar DataFrame
df_categoricas_ordenado = pd.DataFrame(estatisticas_categoricas_detalhadas)


## Analise

In [18]:
print("Análise Descritiva do Conjunto de Dados Numéricos")
print(df_estatisticas_numericas)

print("Análise Descritiva do Conjunto de Dados Categóricos")


Análise Descritiva do Conjunto de Dados Numéricos
    Coluna  Mínimo  Máximo   Média  Desvio Padrão  Dados Faltantes  \
0    V0022     1.0    23.0    3.34           1.52                0   
1   A02305     0.0    36.0    0.68           1.39            21404   
2   A02306     0.0    21.0    1.50           1.46            21404   
3     C008    15.0    50.0   34.23           9.58                0   
4   P00104    30.0   176.0   72.60          15.68              882   
5   P00404   129.0   206.0  166.24           9.70              882   
6     P035     0.0     7.0    3.22           1.91            29240   
7   P03701     0.0    15.0    1.12           0.86            30147   
8   P03702     0.0    59.0   12.90          16.54            30147   
9     P053     3.0    50.0   16.57           4.95            39313   
10  P05901     0.0    47.0   11.38           9.05            43625   
11  P05902     0.0    11.0    1.13           2.52            43625   
12  P05903     0.0     3.0    0.19      

## Salvando em arquivo

In [19]:
doc = Document()
doc.add_heading('Análise Descritiva do Conjunto de Dados', 0)

In [20]:
doc.add_heading('Estatísticas Numéricas', level=1)
tabela_numericas = doc.add_table(rows=1, cols=len(df_estatisticas_numericas.columns))

In [21]:
hdr_cells = tabela_numericas.rows[0].cells

for i, col in enumerate(df_estatisticas_numericas.columns):
    hdr_cells[i].text = str(col)


In [22]:
for index, row in df_estatisticas_numericas.iterrows():
    row_cells = tabela_numericas.add_row().cells
    for i, value in enumerate(row):
        row_cells[i].text = str(value)

In [23]:
doc.add_heading('Análise Categórica', level=1)

for coluna in colunas_categoricas:
    doc.add_heading(f'Coluna: {coluna}', level=2)
    dados_coluna = df_categoricas_ordenado[df_categoricas_ordenado['Coluna'] == coluna]
    
    tabela = doc.add_table(rows=1, cols=3)
    hdr_cells = tabela.rows[0].cells
    hdr_cells[0].text = 'Categoria'
    hdr_cells[1].text = 'Contagem'
    hdr_cells[2].text = '% da Coluna'
    
    for _, row in dados_coluna.iterrows():
        row_cells = tabela.add_row().cells
        row_cells[0].text = str(row['Categoria'])
        row_cells[1].text = str(row['Contagem'])
        row_cells[2].text = f"{row['% da Coluna']}%"

In [24]:
# doc.save('../Relatorios/tabela_estatisticas.docx')
doc.save('../Relatorios/tabela_estatisticas_filtrada.docx')
# doc.save('../Relatorios/tabela_estatisticas_plano_cartesiano.docx')